In [30]:
import ollama

ollama.pull('gemma2:27b')

response = ollama.chat(
    model='gemma2:27b',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}]
)
print(response['message']['content'])


The sky appears blue due to a phenomenon called **Rayleigh scattering**.

Here's how it works:

1. **Sunlight Enters the Atmosphere:** Sunlight is actually white, containing all colors of the rainbow. When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).
2. **Scattering Occurs:** These air molecules are much smaller than the wavelengths of visible light. As a result, they scatter the shorter wavelengths of light (blue and violet) more effectively than longer wavelengths (red, orange, yellow).
3. **Blue Light Dominates:** The scattered blue light travels in many directions, making the sky appear blue to our eyes.

**Why not Violet?**

Even though violet light is scattered even more than blue light, our eyes are less sensitive to violet wavelengths. Therefore, we perceive the sky as predominantly blue.


Let me know if you have any other questions!


In [31]:
import pandas as pd

output_dir = '../relevance'

df = pd.read_csv('../gym/data.csv')

In [32]:
# for each row of the dataframe, we will ask the model to generate a response
# We want to know if the  table is relevant to the queries we have and a score of how relevant it is to the query
# then add to each row the 10 queries as columns and the score of the model for each query


queries = {
    1: '(Graph structured) AND (visual Question Answering)',
    2: '"Autonomous Surface Vehicles" AND "Federated Learning"',
    3: 'Techniques for scene understanding in computer vision',
    4: 'Applications of visual question answering systems',
    5: 'Methods and metrics for image segmentation',
    6: 'Approaches to handwritten text recognition tasks',
    7: 'Techniques for understanding table structures in documents',
    8: 'Advances in speech recognition systems',
    9: 'Developments in machine translation models',
    10: 'Challenges in federated learning applications',
}


def get_prompt(query, table):
    return (f"Given the table [{table}], how relevant is the table to the query: [{query}]?"
            f" please provide a score between 0 and 1 and true or false if the table is relevant to the query,"
            f" response must be in the format: [score|relevant] e.g. 0.9|true")

def get_prompt_response(query, table):
    response = ollama.chat(
        model='gemma2',
        messages=[{'role': 'user', 'content': get_prompt(query, table)}]
    )
    return response['message']['content']


# for each row of the dataframe, we will ask the model to generate a response
# then we export to json the results 
for index, row in df.iterrows():
    for query in queries:
        try:
            response = get_prompt_response(queries[query], row)
            score, relevant = response.split('|')
            print(f"Exporting {index}_{query}.txt")
            with open(f'{output_dir}/{index}_{query}.txt', 'w') as f:
                f.write(response)
        except Exception as e:
            print(e)
            print(response)
            print(queries[query], row)
            break        


Exporting 0_1.txt
Exporting 0_2.txt
Exporting 0_3.txt
Exporting 0_4.txt
Exporting 0_5.txt
Exporting 0_6.txt
Exporting 0_7.txt
Exporting 0_8.txt
Exporting 0_9.txt
Exporting 0_10.txt
Exporting 1_1.txt
Exporting 1_2.txt
Exporting 1_3.txt
Exporting 1_4.txt
Exporting 1_5.txt
Exporting 1_6.txt
Exporting 1_7.txt
Exporting 1_8.txt
Exporting 1_9.txt
Exporting 1_10.txt
Exporting 2_1.txt
Exporting 2_2.txt
Exporting 2_3.txt
Exporting 2_4.txt
Exporting 2_5.txt
Exporting 2_6.txt
Exporting 2_7.txt
Exporting 2_8.txt
Exporting 2_9.txt
Exporting 2_10.txt
Exporting 3_1.txt
Exporting 3_2.txt
Exporting 3_3.txt
Exporting 3_4.txt
Exporting 3_5.txt
Exporting 3_6.txt
Exporting 3_7.txt
Exporting 3_8.txt
Exporting 3_9.txt
Exporting 3_10.txt
Exporting 4_1.txt
Exporting 4_2.txt
Exporting 4_3.txt
Exporting 4_4.txt
Exporting 4_5.txt
Exporting 4_6.txt
Exporting 4_7.txt
Exporting 4_8.txt
Exporting 4_9.txt
Exporting 4_10.txt
Exporting 5_1.txt
Exporting 5_2.txt
Exporting 5_3.txt
Exporting 5_4.txt
Exporting 5_5.txt
Expor